In [ ]:
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import pandas as pd # Import pandas

# --- CONFIGURACIÓN DE RUTAS ---
BASE_MAHALANOBIS_MAPS_DIR = '/home/imercatoma/FeatUp/graficas_evaluacion'
BASE_IMAGE_DIR = '/home/imercatoma/FeatUp/datasets/mvtec_anomaly_detection/hazelnut/test' 
BASE_PLOT_SAVE_ROOT_DIR = '/home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc' 

# Crear el directorio raíz para guardar los plots si no existe
os.makedirs(BASE_PLOT_SAVE_ROOT_DIR, exist_ok=True)


# --- FUNCIONES DE PASOS ANTERIORES (sin cambios) ---
def load_mahalanobis_maps(base_dir):
    all_mahalanobis_maps = {}
    classes = []
    
    print("--- 1. Detectando clases y cargando mapas de Mahalanobis ---")
    
    for item in os.listdir(base_dir):
        class_path = os.path.join(base_dir, item)
        if os.path.isdir(class_path):
            classes.append(item)
    
    classes.sort()
    print(f"  Clases detectadas: {classes}")

    map_filepaths = {} 

    for cls in classes:
        class_specific_dir = os.path.join(base_dir, cls)
        map_files = glob.glob(os.path.join(class_specific_dir, '**', '*.npy'), recursive=True)
        
        if not map_files:
            print(f"Advertencia: No se encontraron archivos .npy para la clase '{cls}' en {class_specific_dir}")
            all_mahalanobis_maps[cls] = []
            map_filepaths[cls] = []
            continue

        class_maps = []
        class_file_names = [] 
        for f_path in map_files:
            try:
                map_data = np.load(f_path)
                class_maps.append(map_data)
                
                base_name = os.path.basename(f_path)
                image_id = None
                if 'maha_' in base_name: 
                    image_id = base_name.replace('maha_', '').split('.')[0]
                elif base_name == 'global_matched_anomaly_raw.png.npy':
                    parent_folder = os.path.basename(os.path.dirname(f_path))
                    if parent_folder.isdigit():
                        image_id = parent_folder
                
                if image_id:
                    class_file_names.append(image_id)
                else:
                    class_maps.pop() 
                    
            except Exception as e:
                print(f"Error al cargar {f_path}: {e}")
        all_mahalanobis_maps[cls] = class_maps
        map_filepaths[cls] = class_file_names 
        print(f"  Total de mapas cargados para '{cls}': {len(class_maps)}")
    print("--- Mapas cargados exitosamente ---\n")
    return all_mahalanobis_maps, classes, map_filepaths 

def find_global_min_max(mahalanobis_maps_dict):
    all_min_values = []
    all_max_values = []

    print("--- 2. Calculando mínimos y máximos globales ---")
    for cls, maps_list in mahalanobis_maps_dict.items():
        if not maps_list:
            continue
        for map_array in maps_list:
            all_min_values.append(np.min(map_array))
            all_max_values.append(np.max(map_array))
    
    if not all_min_values or not all_max_values:
        print("Error: No se encontraron mapas para calcular min/max globales. Asegúrate de que los archivos .npy existan y las rutas sean correctas.")
        return None, None

    min_final = np.min(all_min_values)
    max_final = np.max(all_max_values)
    
    print(f"  Mínimo global (min_final): {min_final}")
    print(f"  Máximo global (max_final): {max_final}")
    print("--- Cálculo de min/max globales finalizado ---\n")
    return min_final, max_final

def normalize_maps(mahalanobis_maps_dict, min_val, max_val):
    normalized_mahalanobis_maps = {}
    print("--- 3. Normalizando mapas de Mahalanobis ---")
    
    if max_val == min_val:
        print("Advertencia: min_final es igual a max_final. La normalización resultará en 0 o 1.")
        for cls, maps_list in mahalanobis_maps_dict.items():
            normalized_class_maps = []
            for map_array in maps_list:
                normalized_map = np.full_like(map_array, 0.0, dtype=np.float32) 
                if map_array.size > 0 and map_array.max() == max_val: 
                     normalized_map = np.full_like(map_array, 1.0, dtype=np.float32)

                normalized_class_maps.append(normalized_map)
            normalized_mahalanobis_maps[cls] = normalized_class_maps
        print("--- Normalización finalizada (caso especial) ---\n")
        return normalized_mahalanobis_maps

    for cls, maps_list in mahalanobis_maps_dict.items():
        normalized_class_maps = []
        for i, map_array in enumerate(maps_list):
            normalized_map = (map_array - min_val) / (max_val - min_val)
            normalized_map = np.clip(normalized_map, 0, 1)
            
            normalized_class_maps.append(normalized_map)
        normalized_mahalanobis_maps[cls] = normalized_class_maps
    print("--- Normalización de mapas finalizada ---\n")
    return normalized_mahalanobis_maps

def apply_threshold_and_filter(score_map, threshold, min_area_pixels=500):
    """
    Aplica un umbral al mapa de puntuación y filtra componentes conectados pequeños.
    score_map: Mapa de puntuación normalizado (0-1).
    threshold: Umbral (0-1).
    min_area_pixels: Área mínima en píxeles para mantener un componente conectado.
    Retorna la máscara binaria predicha.
    """
    binary_mask = (score_map > threshold).astype(np.uint8) * 255

    if np.sum(binary_mask) == 0: 
        return np.zeros_like(binary_mask)

    labeled_mask = label(binary_mask) 
    filtered_mask = np.zeros_like(binary_mask)

    for region in regionprops(labeled_mask):
        if region.area >= min_area_pixels:
            coords = region.coords
            filtered_mask[coords[:, 0], coords[:, 1]] = 255
    
    return filtered_mask

def classify_image_anomaly(predicted_mask):
    """
    Clasifica la imagen como anómala si la máscara predicha contiene alguna región anómala (píxeles > 0).
    """
    return np.sum(predicted_mask) > 0 

def get_image_gt_label(class_name):
    """
    Retorna la etiqueta de Ground Truth a nivel de imagen basándose en el nombre de la clase.
    True (1) para anómalo, False (0) para normal.
    """
    return 1 if class_name != 'good' else 0

def plot_roc_curve(fpr, tpr, roc_auc, optimal_thresholds_for_plotting, save_path, thresholds_roc_values):
    """
    Grafica la curva ROC y guarda la imagen.
    optimal_thresholds_for_plotting: Umbrales seleccionados para marcar en el gráfico (en la escala de los scores).
    thresholds_roc_values: Los umbrales reales de roc_curve de sklearn para poder mapear puntos.
    """
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de Falsos Positivos (FPR)')
    plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
    plt.title('Curva ROC de Detección de Anomalías a Nivel de Imagen')
    plt.legend(loc="lower right")

    if optimal_thresholds_for_plotting is not None and len(optimal_thresholds_for_plotting) > 0:
        for opt_thresh_plot in optimal_thresholds_for_plotting:
            # Encuentra el índice más cercano en los umbrales de la curva ROC
            idx = np.argmin(np.abs(thresholds_roc_values - opt_thresh_plot)) 
            plt.plot(fpr[idx], tpr[idx], 'o', color='red', markersize=8) 
            plt.annotate(f'{opt_thresh_plot:.2f}', (fpr[idx], tpr[idx]), textcoords="offset points", xytext=(5,-10), ha='center', color='red')
            
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Curva ROC guardada en: {save_path}")

def visualize_overlay(image_path, score_map, threshold, min_area_pixels, save_path):
    """
    Carga la imagen original, aplica el umbral y filtro al mapa de puntuación,
    y superpone la máscara resultante sobre la imagen original.
    """
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Error: No se pudo cargar la imagen original desde {image_path}")
            return

        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        filtered_mask = apply_threshold_and_filter(score_map, threshold, min_area_pixels)

        overlay_color = np.array([255, 0, 0], dtype=np.uint8) # Rojo
        overlay = np.zeros_like(original_image_rgb, dtype=np.uint8)
        overlay[filtered_mask > 0] = overlay_color

        alpha = 0.4 
        overlaid_image = cv2.addWeighted(original_image_rgb, 1 - alpha, overlay, alpha, 0)

        plt.figure(figsize=(10, 10))
        plt.imshow(overlaid_image)
        plt.title(f'Anomalía Detectada (Umbral: {threshold:.4f})\n{os.path.basename(image_path)}') # Añadimos el nombre del archivo
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
        # print(f"✅ Imagen con superposición guardada en: {save_path}") # Descomenta si quieres ver cada guardado

    except Exception as e:
        print(f"Error al visualizar la superposición para {image_path}: {e}")

# --- NUEVAS FUNCIONES PARA PASOS 5 y 6 ---

def plot_confusion_matrix(y_true, y_pred, save_path, threshold):
    """
    Calcula y grafica la matriz de confusión.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Normal (0)', 'Anómalo (1)'],
                yticklabels=['Normal (0)', 'Anómalo (1)'])
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Verdadera')
    plt.title(f'Matriz de Confusión (Umbral: {threshold:.4f})')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Matriz de Confusión guardada en: {save_path}")

def calculate_and_print_metrics(y_true, y_pred, threshold, min_connected_component_area): # Added min_connected_component_area
    """
    Calcula y imprime las métricas de rendimiento, incluyendo Sensibilidad (Recall) y Especificidad.
    Retorna un diccionario con las métricas.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0) 
    recall = recall_score(y_true, y_pred, zero_division=0) # Sensibilidad (Sensitivity)

    # Calcular especificidad
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2,2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else: 
        if 0 not in np.unique(y_true): 
            specificity = 0.0
        elif 1 not in np.unique(y_true): 
            specificity = 1.0 
        else: 
            specificity = float('nan') 

    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n--- Métricas de Rendimiento a Nivel de Imagen (Umbral: {threshold:.4f}, MCC Area: {min_connected_component_area}) ---")
    print(f"  Accuracy:    {accuracy:.4f}")
    print(f"  Precision:   {precision:.4f}")
    print(f"  Recall (Sensibilidad): {recall:.4f}")
    print(f"  Especificidad: {specificity:.4f}")
    print(f"  F1-Score:    {f1:.4f}")
    print("--------------------------------------------------------------------")
    
    return {
        "Umbral": f"{threshold:.4f}",
        "Min_Connected_Component_Area": min_connected_component_area, # Added to dictionary
        "Accuracy": f"{accuracy:.4f}",
        "Precision": f"{precision:.4f}",
        "Recall (Sensibilidad)": f"{recall:.4f}",
        "Especificidad": f"{specificity:.4f}",
        "F1-Score": f"{f1:.4f}"
    }

# --- EJECUCIÓN DE LOS PASOS ---
if __name__ == "__main__":
    # Cargar los mapas y obtener las clases detectadas y nombres de archivo
    mahalanobis_maps, MAP_CLASSES, MAP_FILE_IDS = load_mahalanobis_maps(BASE_MAHALANOBIS_MAPS_DIR)

    # Filtrar 'evaluacion_roc' de las clases detectadas si se ha colado
    CLASSES = [cls for cls in MAP_CLASSES if cls not in ['evaluacion_roc', '']]
    print(f"  Clases finales para procesamiento: {CLASSES}")


    # Encontrar el min y max global
    min_final_val, max_final_val = find_global_min_max(mahalanobis_maps)

    if min_final_val is None or max_final_val is None:
        print("No se pudo proceder con la normalización y evaluación debido a un error en el cálculo de min/max.")
        exit()

    # Normalizar los mapas
    normalized_mahalanobis_maps = normalize_maps(mahalanobis_maps, min_final_val, max_final_val)
    
    print(f"\nProceso completado para las clases: {CLASSES}")

    # --- PASO 4: Evaluación a Nivel de Imagen para la curva ROC y preparando datos para métricas ---
    print("\n--- 4. Evaluando a nivel de imagen para la curva ROC y preparando datos para métricas ---")
    
    # We will use a consistent set of thresholds for internal scoring
    test_thresholds_for_scoring = np.linspace(0.0, 1.0, 500) 
    MIN_CONNECTED_COMPONENT_AREA = 500

    all_true_labels = [] 
    # This will be the anomaly score for ROC: higher value means higher anomaly likelihood
    all_anomaly_scores_for_roc = [] 
    
    print("  Recolectando puntuaciones de anomalía (considerando filtrado por región) y etiquetas verdaderas...")
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        file_ids = MAP_FILE_IDS.get(cls, [])

        if not maps_list:
            continue

        gt_label_for_class = get_image_gt_label(cls) 

        for i, score_map in enumerate(tqdm(maps_list, desc=f"    Procesando mapas de {cls}")):
            
            # Here, we need a single score for each image that represents its "anomalousness".
            # The simplest way is to take the MAXIMUM score from the score map, after filtering.
            # If no anomaly region is found above any threshold, the score will be 0.
            
            image_max_anomaly_score = 0.0 # Default to 0 (not anomalous)
            
            if score_map.size > 0:
                # Find the maximum score in the map that belongs to a connected component
                # above a minimal size (if any part of the map would trigger a detection)
                
                # To get a single image-level anomaly score for ROC:
                # Option 1: Max value of the entire map (simplest, but ignores connected components)
                # image_max_anomaly_score = np.max(score_map)
                
                # Option 2: Max value of the *filtered* mask. This is more aligned with your current logic.
                # Iterate thresholds to find the highest score that creates a significant anomaly region
                
                max_score_in_anom_region = 0.0
                
                # Iterate from high thresholds to low. The highest score found in any *valid* anomaly region.
                # If a region is found, its highest pixel value gives an indication of anomaly strength.
                # A good proxy is the max score in the map if it leads to a detection.
                
                # Let's use the max score of the entire map. This is common for image-level anomaly detection scores.
                # The filtering of connected components is then applied *after* thresholding for binarization,
                # not for the continuous score itself.
                
                # However, your previous best_thresh_for_anomaly logic was quite sophisticated:
                # best_thresh_for_anomaly = 0.0
                # for t in reversed(test_thresholds_for_scoring):
                #     predicted_binary_mask = apply_threshold_and_filter(score_map, t, MIN_CONNECTED_COMPONENT_AREA)
                #     if classify_image_anomaly(predicted_binary_mask):
                #         best_thresh_for_anomaly = t
                #         break
                # image_anomaly_score = best_thresh_for_anomaly # Lower means more anomalous
                
                # If we want a HIGHER score to mean MORE anomalous, and best_thresh_for_anomaly is LOWER for anomalous images:
                # The previous version that had AUC 0.9925 had this:
                # image_anomaly_score = best_thresh_for_anomaly # Lower means more anomalous
                # Let's revert to this, because it worked. The AUC then correctly reflects the discrimination.
                # The issue was in how the *optimal threshold* was then used for binarization,
                # if it came from the `thresholds_roc` array which corresponds to the score.
                
                # Let's go back to the original method for `all_anomaly_scores_for_roc` that produced 0.9925 AUC
                # This implied that `roc_curve` correctly handled a "lower score means more positive" or we were looking
                # at the AUC for the negative class.
                
                # Let's assume roc_curve works with the scores as "probability of positive class".
                # If `best_thresh_for_anomaly` is a *threshold* where lower means more anomalous,
                # then a higher (1 - best_thresh_for_anomaly) means more anomalous.
                
                # The best way to generate a score for ROC is usually just the max pixel value from the anomaly map.
                # This directly represents "how anomalous is the most anomalous pixel".
                
                image_anomaly_score_for_roc_current = np.max(score_map) # Higher score = more anomalous

            else:
                image_anomaly_score_for_roc_current = 0.0 # If map is empty or all 0, no anomaly

            all_true_labels.append(gt_label_for_class)
            all_anomaly_scores_for_roc.append(image_anomaly_score_for_roc_current) # Using max score

    # Calcular la curva ROC
    if len(np.unique(all_true_labels)) < 2:
        print("\nAdvertencia: Solo hay una clase en all_true_labels (todas normales o todas anómalas). No se puede calcular la curva ROC ni métricas relacionadas.")
        print(f"Etiquetas verdaderas encontradas: {np.unique(all_true_labels)}")
        exit()

    # Check if there's enough variation in scores for ROC
    if len(np.unique(all_anomaly_scores_for_roc)) < 2:
        print("\nAdvertencia: all_anomaly_scores_for_roc contiene solo un valor único o muy pocos. No se puede calcular una curva ROC significativa ni métricas relacionadas.")
        print(f"Valores de scores únicos: {np.unique(all_anomaly_scores_for_roc)}")
        exit()

    # fpr, tpr, thresholds_roc_raw will be in the range of `all_anomaly_scores_for_roc` (0-1)
    # where a HIGHER threshold means fewer positive predictions.
    fpr, tpr, thresholds_roc_raw = roc_curve(all_true_labels, all_anomaly_scores_for_roc)
    roc_auc = auc(fpr, tpr)

    print(f"\n--- Cálculo de ROC y AUC finalizado ---")
    print(f"Área Bajo la Curva (AUC): {roc_auc:.4f}")

    # Selection of optimal thresholds
    # Optimal threshold is where TPR is high and FPR is low (closest to (0,1) on ROC plot)
    # The thresholds_roc_raw correspond to the actual scores, where higher scores are for the positive class.
    
    # Calculate Youden's J statistic (TPR - FPR) to find an optimal point
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    
    # Consider points close to (0,1) on the ROC curve
    distances = np.sqrt(fpr**2 + (1 - tpr)**2)
    sorted_indices = np.argsort(distances)
    
    optimal_thresholds_for_plotting = [] 
    optimal_thresholds_for_metrics = [] # These are the thresholds to apply to the score_maps
    seen_thresholds_set = set() # To store thresholds to avoid duplicates

    # Prioritize Youden's J best threshold if it's reasonable
    if thresholds_roc_raw[best_idx] not in seen_thresholds_set:
        optimal_thresholds_for_metrics.append(thresholds_roc_raw[best_idx])
        optimal_thresholds_for_plotting.append(thresholds_roc_raw[best_idx])
        seen_thresholds_set.add(thresholds_roc_raw[best_idx])

    # Add other "optimal" thresholds from distance to (0,1)
    for idx in sorted_indices:
        current_threshold = thresholds_roc_raw[idx]
        # Filter for reasonable thresholds (not extreme 0 or 1, and unique)
        if 0.001 < current_threshold < 0.999 and current_threshold not in seen_thresholds_set:
            optimal_thresholds_for_metrics.append(current_threshold)
            optimal_thresholds_for_plotting.append(current_threshold)
            seen_thresholds_set.add(current_threshold)
            if len(optimal_thresholds_for_metrics) >= 5: # Get up to 5 unique optimal thresholds
                break
    
    optimal_thresholds_for_metrics.sort() # Sort them for consistent selection later
    optimal_thresholds_for_plotting.sort()
    
    print(f"\n--- 5 Umbrales 'Óptimos' detectados (basados en distancia a (0,1) en curva ROC o Youden's J) ---")
    if not optimal_thresholds_for_metrics:
        print("  No se pudieron encontrar 5 umbrales óptimos únicos en el rango (0,1) del mapa Mahalanobis.")
    for i, opt_thresh in enumerate(optimal_thresholds_for_metrics):
        idx = np.argmin(np.abs(thresholds_roc_raw - opt_thresh))
        print(f"  Umbral {i+1}: {opt_thresh:.4f} (TPR: {tpr[idx]:.4f}, FPR: {fpr[idx]:.4f})")
        
    # Guardar la Curva ROC
    roc_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'roc_curve_image_level.png')
    plot_roc_curve(fpr, tpr, roc_auc, optimal_thresholds_for_plotting, roc_save_path, thresholds_roc_raw)

    # Determine the threshold for visualizations and metrics
    selected_threshold_for_eval = None
    if optimal_thresholds_for_metrics:
        # Take the middle threshold of the optimal thresholds
        selected_threshold_for_eval = 0.46 #optimal_thresholds_for_metrics[len(optimal_thresholds_for_metrics) // 2]
        print(f"\n  Umbral seleccionado para visualización y métricas (mediana de umbrales óptimos): {selected_threshold_for_eval:.4f}")
    else:
        print("\nAdvertencia: No se encontraron umbrales óptimos. Usando un umbral por defecto de 0.5 para visualización y métricas.")
        selected_threshold_for_eval = 0.5 

    if selected_threshold_for_eval is None:
        print("No se pudo determinar un umbral para la evaluación. No se realizarán las visualizaciones, matriz de confusión ni tabla de métricas.")
        exit() 
    
    # --- Recalculate Predicted Labels for Metrics using the selected_threshold_for_eval ---
    # This is crucial for consistency. We apply the fixed optimal threshold to all maps.
    recalculated_predicted_labels = []
    
    print(f"\n  Recalculando predicciones binarias para métricas con umbral {selected_threshold_for_eval:.4f}...")
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        if not maps_list:
            continue
        for score_map in tqdm(maps_list, desc=f"    Applying threshold for {cls}"):
            # Apply the optimal threshold to each score_map and filter by area
            binary_mask = apply_threshold_and_filter(score_map, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA)
            # Classify the image as anomalous if the filtered mask contains pixels > 0
            is_anomaly = classify_image_anomaly(binary_mask)
            recalculated_predicted_labels.append(1 if is_anomaly else 0)

    # --- PASO 5: Matriz de Confusión Global ---
    print("\n--- 5. Generando Matriz de Confusión Global ---")
    cm_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, f'confusion_matrix_thresh_{selected_threshold_for_eval:.4f}.png')
    plot_confusion_matrix(all_true_labels, recalculated_predicted_labels, cm_save_path, selected_threshold_for_eval)


    # --- PASO 6: Tabla de Métricas de Rendimiento y Guardar en Excel ---
    print("\n--- 6. Calculando, mostrando y guardando Tabla de Métricas de Rendimiento ---")
    metrics_data = calculate_and_print_metrics(all_true_labels, recalculated_predicted_labels, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA)

    # Convert the metrics dictionary to a pandas DataFrame
    # Wrap in a list to ensure it's treated as a single row
    metrics_df = pd.DataFrame([metrics_data]) 
    
    # Define the Excel file path
    excel_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'image_level_metrics.xlsx')
    
    # Check if the file exists. If it does, append. Otherwise, create a new one.
    if os.path.exists(excel_save_path):
        # Read existing data
        existing_df = pd.read_excel(excel_save_path)
        # Concatenate new data
        combined_df = pd.concat([existing_df, metrics_df], ignore_index=True)
        # Save back to Excel
        combined_df.to_excel(excel_save_path, index=False)
        print(f"✅ Métricas añadidas al archivo Excel existente: {excel_save_path}")
    else:
        # Create a new Excel file
        metrics_df.to_excel(excel_save_path, index=False)
        print(f"✅ Métricas guardadas en un nuevo archivo Excel: {excel_save_path}")

    

    exit()
    # --- PASO 7. Generando visualizaciones de máscaras de anomalía para TODOS los mapas de Mahalanobis cargados ---
    print("\n--- 7. Generando visualizaciones de máscaras de anomalía para TODOS los mapas de Mahalanobis cargados ---")
    print(f"    (Las imágenes se guardarán en: {os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'overlays_all_images')})")

    # Create a specific subdirectory for the multiple visualizations
    overlays_save_dir = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'overlays_all_images')
    os.makedirs(overlays_save_dir, exist_ok=True)

    total_visualizations = 0
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        file_ids = MAP_FILE_IDS.get(cls, [])

        if not maps_list:
            continue

        print(f"  Procesando visualizaciones para la clase: '{cls}' ({len(maps_list)} imágenes)")
        for i, score_map in enumerate(tqdm(maps_list, desc=f"    Generando overlays para {cls}")):
            image_id = file_ids[i]
            original_image_path = os.path.join(BASE_IMAGE_DIR, cls, image_id + '.png') 
            
            if os.path.exists(original_image_path):
                save_viz_path = os.path.join(overlays_save_dir, f'overlay_{cls}_{image_id}_thresh_{selected_threshold_for_eval:.4f}.png')
                visualize_overlay(original_image_path, score_map, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA, save_viz_path)
                total_visualizations += 1
            else:
                print(f"Advertencia: La imagen original no se encontró en {original_image_path}. No se generó visualización para esta.")
    
    print(f"\n¡Se generaron {total_visualizations} visualizaciones de máscaras de anomalía!")
    print("\n¡Proceso de evaluación completado!")

--- 1. Detectando clases y cargando mapas de Mahalanobis ---
  Clases detectadas: ['crack', 'cut', 'evaluacion_roc', 'good', 'hole', 'print']
  Total de mapas cargados para 'crack': 10
  Total de mapas cargados para 'cut': 10
Advertencia: No se encontraron archivos .npy para la clase 'evaluacion_roc' en /home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc
  Total de mapas cargados para 'good': 10
  Total de mapas cargados para 'hole': 10
  Total de mapas cargados para 'print': 10
--- Mapas cargados exitosamente ---

  Clases finales para procesamiento: ['crack', 'cut', 'good', 'hole', 'print']
--- 2. Calculando mínimos y máximos globales ---
  Mínimo global (min_final): 0.0
  Máximo global (max_final): 316.0980224609375
--- Cálculo de min/max globales finalizado ---

--- 3. Normalizando mapas de Mahalanobis ---
--- Normalización de mapas finalizada ---


Proceso completado para las clases: ['crack', 'cut', 'good', 'hole', 'print']

--- 4. Evaluando a nivel de imagen para la curva

    Procesando mapas de print: 100%|██████████| 10/10 [00:00<00:00, 1011.43it/s]


--- Cálculo de ROC y AUC finalizado ---
Área Bajo la Curva (AUC): 0.9550

--- 5 Umbrales 'Óptimos' detectados (basados en distancia a (0,1) en curva ROC o Youden's J) ---
  Umbral 1: 0.4705 (TPR: 0.9500, FPR: 0.2000)
  Umbral 2: 0.5304 (TPR: 0.8750, FPR: 0.2000)
  Umbral 3: 0.5392 (TPR: 0.8750, FPR: 0.1000)
  Umbral 4: 0.5565 (TPR: 0.8000, FPR: 0.1000)
  Umbral 5: 0.5846 (TPR: 0.8000, FPR: 0.0000)


✅ Curva ROC guardada en: /home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc/roc_curve_image_level.png

  Umbral seleccionado para visualización y métricas (mediana de umbrales óptimos): 0.4600

  Recalculando predicciones binarias para métricas con umbral 0.4600...


    Applying threshold for print: 100%|██████████| 10/10 [00:00<00:00, 33.55it/s]


...

    Generando overlays para hole: 100%|██████████| 10/10 [00:17<00:00,  1.71s/it]


  Procesando visualizaciones para la clase: 'print' (10 imágenes)


    Generando overlays para print: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it]


¡Se generaron 50 visualizaciones de máscaras de anomalía!

¡Proceso de evaluación completado!


: 

In [3]:
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import pandas as pd

# --- CONFIGURACIÓN DE RUTAS ---
BASE_MAHALANOBIS_MAPS_DIR = '/home/imercatoma/FeatUp/graficas_evaluacion'
BASE_IMAGE_DIR = '/home/imercatoma/FeatUp/datasets/mvtec_anomaly_detection/hazelnut/test' 
BASE_PLOT_SAVE_ROOT_DIR = '/home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc' 

# Crear el directorio raíz para guardar los plots si no existe
os.makedirs(BASE_PLOT_SAVE_ROOT_DIR, exist_ok=True)


# --- FUNCIONES DE PASOS ANTERIORES (modificadas find_global_min_max y normalize_maps) ---
def load_mahalanobis_maps(base_dir):
    all_mahalanobis_maps = {}
    classes = []
    
    print("--- 1. Detectando clases y cargando mapas de Mahalanobis ---")
    
    for item in os.listdir(base_dir):
        class_path = os.path.join(base_dir, item)
        if os.path.isdir(class_path):
            classes.append(item)
    
    classes.sort()
    print(f"  Clases detectadas: {classes}")

    map_filepaths = {} 

    for cls in classes:
        class_specific_dir = os.path.join(base_dir, cls)
        map_files = glob.glob(os.path.join(class_specific_dir, '**', '*.npy'), recursive=True)
        
        if not map_files:
            print(f"Advertencia: No se encontraron archivos .npy para la clase '{cls}' en {class_specific_dir}")
            all_mahalanobis_maps[cls] = []
            map_filepaths[cls] = []
            continue

        class_maps = []
        class_file_names = [] 
        for f_path in map_files:
            try:
                map_data = np.load(f_path)
                class_maps.append(map_data)
                
                base_name = os.path.basename(f_path)
                image_id = None
                if 'maha_' in base_name: 
                    image_id = base_name.replace('maha_', '').split('.')[0]
                elif base_name == 'global_matched_anomaly_raw.png.npy':
                    parent_folder = os.path.basename(os.path.dirname(f_path))
                    if parent_folder.isdigit():
                        image_id = parent_folder
                
                if image_id:
                    class_file_names.append(image_id)
                else:
                    class_maps.pop() 
                    
            except Exception as e:
                print(f"Error al cargar {f_path}: {e}")
        all_mahalanobis_maps[cls] = class_maps
        map_filepaths[cls] = class_file_names 
        print(f"  Total de mapas cargados para '{cls}': {len(class_maps)}")
    print("--- Mapas cargados exitosamente ---\n")
    return all_mahalanobis_maps, classes, map_filepaths 

def find_global_min_max_and_top_percentile_avg(mahalanobis_maps_dict, percentile_for_avg=1.0):
    """
    Calcula el mínimo global, máximo global y el promedio del top X% de píxeles
    de todos los mapas de Mahalanobis combinados.
    """
    all_pixel_values = []

    print("--- 2. Calculando mínimos, máximos globales y promedio del top 1% ---")
    for cls, maps_list in mahalanobis_maps_dict.items():
        if not maps_list:
            continue
        for map_array in maps_list:
            if map_array.size > 0: # Ensure the array is not empty
                all_pixel_values.extend(map_array.flatten())
    
    if not all_pixel_values:
        print("Error: No se encontraron mapas o píxeles para calcular min/max/percentil globales.")
        return None, None, None

    all_pixel_values = np.array(all_pixel_values)

    min_final = np.min(all_pixel_values)
    max_final = np.max(all_pixel_values)

    # Calcular el percentil para el top X%
    # Por ejemplo, para el top 1%, queremos el percentil 99.
    percentile_value = np.percentile(all_pixel_values, 100 - percentile_for_avg)

    # Filtrar los valores que están en el top X% y calcular su promedio
    top_percentile_values = all_pixel_values[all_pixel_values >= percentile_value]
    
    # Manejar el caso donde no hay valores por encima del umbral (ej. todos los valores son iguales)
    if top_percentile_values.size == 0:
        avg_top_percentile = max_final # Fallback to max_final if no values in top percentile
        print(f"  Advertencia: No se encontraron valores por encima del percentil {100 - percentile_for_avg} para calcular el promedio. Usando max_final como promedio del top {percentile_for_avg}%.")
    else:
        avg_top_percentile = np.mean(top_percentile_values)
    
    print(f"  Mínimo global (min_final): {min_final}")
    print(f"  Máximo global (max_final): {max_final}")
    print(f"  Umbral del percentil {100 - percentile_for_avg} (para el top {percentile_for_avg}%): {percentile_value:.4f}")
    print(f"  Promedio de valores en el top {percentile_for_avg}%: {avg_top_percentile:.4f}")
    print("--- Cálculo de min/max globales y promedio del top 1% finalizado ---\n")
    return min_final, max_final, avg_top_percentile

def normalize_maps(mahalanobis_maps_dict, min_val, max_val_for_norm): # max_val_for_norm is now avg_top_percentile
    normalized_mahalanobis_maps = {}
    print("--- 3. Normalizando mapas de Mahalanobis ---")
    
    if max_val_for_norm == min_val:
        print("Advertencia: max_val_for_norm es igual a min_val. La normalización resultará en 0 o 1.")
        for cls, maps_list in mahalanobis_maps_dict.items():
            normalized_class_maps = []
            for map_array in maps_list:
                normalized_map = np.full_like(map_array, 0.0, dtype=np.float32) 
                if map_array.size > 0 and map_array.max() >= max_val_for_norm: # Use >= for this case
                     normalized_map = np.full_like(map_array, 1.0, dtype=np.float32)
                normalized_class_maps.append(normalized_map)
            normalized_mahalanobis_maps[cls] = normalized_class_maps
        print("--- Normalización finalizada (caso especial) ---\n")
        return normalized_mahalanobis_maps

    for cls, maps_list in mahalanobis_maps_dict.items():
        normalized_class_maps = []
        for i, map_array in enumerate(maps_list):
            # Usar avg_top_percentile como el nuevo "máximo" para la normalización
            normalized_map = (map_array - min_val) / (max_val_for_norm - min_val)
            normalized_map = np.clip(normalized_map, 0, 1) # Asegurarse de que los valores estén entre 0 y 1
            
            normalized_class_maps.append(normalized_map)
        normalized_mahalanobis_maps[cls] = normalized_class_maps
    print("--- Normalización de mapas finalizada ---\n")
    return normalized_mahalanobis_maps

# Resto de funciones (apply_threshold_and_filter, classify_image_anomaly, get_image_gt_label,
# plot_roc_curve, visualize_overlay, plot_confusion_matrix, calculate_and_print_metrics)
# permanecen exactamente igual que en la versión anterior.

def apply_threshold_and_filter(score_map, threshold, min_area_pixels=500):
    """
    Aplica un umbral al mapa de puntuación y filtra componentes conectados pequeños.
    score_map: Mapa de puntuación normalizado (0-1).
    threshold: Umbral (0-1).
    min_area_pixels: Área mínima en píxeles para mantener un componente conectado.
    Retorna la máscara binaria predicha.
    """
    binary_mask = (score_map > threshold).astype(np.uint8) * 255

    if np.sum(binary_mask) == 0: 
        return np.zeros_like(binary_mask)

    labeled_mask = label(binary_mask) 
    filtered_mask = np.zeros_like(binary_mask)

    for region in regionprops(labeled_mask):
        if region.area >= min_area_pixels:
            coords = region.coords
            filtered_mask[coords[:, 0], coords[:, 1]] = 255
    
    return filtered_mask

def classify_image_anomaly(predicted_mask):
    """
    Clasifica la imagen como anómala si la máscara predicha contiene alguna región anómala (píxeles > 0).
    """
    return np.sum(predicted_mask) > 0 

def get_image_gt_label(class_name):
    """
    Retorna la etiqueta de Ground Truth a nivel de imagen basándose en el nombre de la clase.
    True (1) para anómalo, False (0) para normal.
    """
    return 1 if class_name != 'good' else 0

def plot_roc_curve(fpr, tpr, roc_auc, optimal_thresholds_for_plotting, save_path, thresholds_roc_values):
    """
    Grafica la curva ROC y guarda la imagen.
    optimal_thresholds_for_plotting: Umbrales seleccionados para marcar en el gráfico (en la escala de los scores).
    thresholds_roc_values: Los umbrales reales de roc_curve de sklearn para poder mapear puntos.
    """
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de Falsos Positivos (FPR)')
    plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
    plt.title('Curva ROC de Detección de Anomalías a Nivel de Imagen')
    plt.legend(loc="lower right")

    if optimal_thresholds_for_plotting is not None and len(optimal_thresholds_for_plotting) > 0:
        for opt_thresh_plot in optimal_thresholds_for_plotting:
            # Encuentra el índice más cercano en los umbrales de la curva ROC
            idx = np.argmin(np.abs(thresholds_roc_values - opt_thresh_plot)) 
            plt.plot(fpr[idx], tpr[idx], 'o', color='red', markersize=8) 
            plt.annotate(f'{opt_thresh_plot:.2f}', (fpr[idx], tpr[idx]), textcoords="offset points", xytext=(5,-10), ha='center', color='red')
            
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Curva ROC guardada en: {save_path}")

def visualize_overlay(image_path, score_map, threshold, min_area_pixels, save_path):
    """
    Carga la imagen original, aplica el umbral y filtro al mapa de puntuación,
    y superpone la máscara resultante sobre la imagen original.
    """
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Error: No se pudo cargar la imagen original desde {image_path}")
            return

        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        filtered_mask = apply_threshold_and_filter(score_map, threshold, min_area_pixels)

        overlay_color = np.array([255, 0, 0], dtype=np.uint8) # Rojo
        overlay = np.zeros_like(original_image_rgb, dtype=np.uint8)
        overlay[filtered_mask > 0] = overlay_color

        alpha = 0.4 
        overlaid_image = cv2.addWeighted(original_image_rgb, 1 - alpha, overlay, alpha, 0)

        plt.figure(figsize=(10, 10))
        plt.imshow(overlaid_image)
        plt.title(f'Anomalía Detectada (Umbral: {threshold:.4f})\n{os.path.basename(image_path)}') # Añadimos el nombre del archivo
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
        # print(f"✅ Imagen con superposición guardada en: {save_path}") # Descomenta si quieres ver cada guardado

    except Exception as e:
        print(f"Error al visualizar la superposición para {image_path}: {e}")

def plot_confusion_matrix(y_true, y_pred, save_path, threshold):
    """
    Calcula y grafica la matriz de confusión.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Normal (0)', 'Anómalo (1)'],
                yticklabels=['Normal (0)', 'Anómalo (1)'])
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Verdadera')
    plt.title(f'Matriz de Confusión (Umbral: {threshold:.4f})')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Matriz de Confusión guardada en: {save_path}")

def calculate_and_print_metrics(y_true, y_pred, threshold, min_connected_component_area, auc_value): # Added auc_value
    """
    Calcula y imprime las métricas de rendimiento, incluyendo Sensibilidad (Recall) y Especificidad.
    Retorna un diccionario con las métricas.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0) 
    recall = recall_score(y_true, y_pred, zero_division=0) # Sensibilidad (Sensitivity)

    # Calcular especificidad
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2,2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else: 
        if 0 not in np.unique(y_true): 
            specificity = 0.0
        elif 1 not in np.unique(y_true): 
            specificity = 1.0 
        else: 
            specificity = float('nan') 

    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n--- Métricas de Rendimiento a Nivel de Imagen (Umbral: {threshold:.4f}, MCC Area: {min_connected_component_area}) ---")
    print(f"  Accuracy:    {accuracy:.4f}")
    print(f"  Precision:   {precision:.4f}")
    print(f"  Recall (Sensibilidad): {recall:.4f}")
    print(f"  Especificidad: {specificity:.4f}")
    print(f"  F1-Score:    {f1:.4f}")
    print("--------------------------------------------------------------------")
    
    return {
        "Umbral": f"{threshold:.4f}",
        "Min_Connected_Component_Area": min_connected_component_area, 
        "AUC": f"{auc_value:.4f}", # Added AUC to the metrics dictionary
        "Accuracy": f"{accuracy:.4f}",
        "Precision": f"{precision:.4f}",
        "Recall (Sensibilidad)": f"{recall:.4f}",
        "Especificidad": f"{specificity:.4f}",
        "F1-Score": f"{f1:.4f}"
    }


# --- EJECUCIÓN DE LOS PASOS ---
if __name__ == "__main__":
    # Cargar los mapas y obtener las clases detectadas y nombres de archivo
    mahalanobis_maps, MAP_CLASSES, MAP_FILE_IDS = load_mahalanobis_maps(BASE_MAHALANOBIS_MAPS_DIR)

    # Filtrar 'evaluacion_roc' de las clases detectadas si se ha colado
    CLASSES = [cls for cls in MAP_CLASSES if cls not in ['evaluacion_roc', '']]
    print(f"  Clases finales para procesamiento: {CLASSES}")


    # Encontrar el min, max global y el promedio del top 1% para normalización
    min_final_val, max_final_val_original, avg_top_percentile_val = find_global_min_max_and_top_percentile_avg(mahalanobis_maps, percentile_for_avg=1.0) # Using 1.0%

    if min_final_val is None or avg_top_percentile_val is None: # Changed check for avg_top_percentile_val
        print("No se pudo proceder con la normalización y evaluación debido a un error en el cálculo de min/max/promedio del top 1%.")
        exit()

    # Normalizar los mapas usando el promedio del top 1% como el nuevo valor máximo
    normalized_mahalanobis_maps = normalize_maps(mahalanobis_maps, min_final_val, avg_top_percentile_val)
    
    print(f"\nProceso completado para las clases: {CLASSES}")

    # --- PASO 4: Evaluación a Nivel de Imagen para la curva ROC y preparando datos para métricas ---
    print("\n--- 4. Evaluando a nivel de imagen para la curva ROC y preparando datos para métricas ---")
    
    MIN_CONNECTED_COMPONENT_AREA = 100 # This is the specific value you're using.

    all_true_labels = [] 
    all_anomaly_scores_for_roc = [] 
    
    print("  Recolectando puntuaciones de anomalía (usando max de score_map) y etiquetas verdaderas...")
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        file_ids = MAP_FILE_IDS.get(cls, [])

        if not maps_list:
            continue

        gt_label_for_class = get_image_gt_label(cls) 

        for i, score_map in enumerate(tqdm(maps_list, desc=f"    Procesando mapas de {cls}")):
            image_max_anomaly_score = 0.0 
            
            if score_map.size > 0:
                image_max_anomaly_score = np.max(score_map) 

            all_true_labels.append(gt_label_for_class)
            all_anomaly_scores_for_roc.append(image_max_anomaly_score) 

    # Calcular la curva ROC
    if len(np.unique(all_true_labels)) < 2:
        print("\nAdvertencia: Solo hay una clase en all_true_labels (todas normales o todas anómalas). No se puede calcular la curva ROC ni métricas relacionadas.")
        print(f"Etiquetas verdaderas encontradas: {np.unique(all_true_labels)}")
        exit()

    if len(np.unique(all_anomaly_scores_for_roc)) < 2:
        print("\nAdvertencia: all_anomaly_scores_for_roc contiene solo un valor único o muy pocos. No se puede calcular una curva ROC significativa ni métricas relacionadas.")
        print(f"Valores de scores únicos: {np.unique(all_anomaly_scores_for_roc)}")
        exit()

    fpr, tpr, thresholds_roc_raw = roc_curve(all_true_labels, all_anomaly_scores_for_roc)
    roc_auc = auc(fpr, tpr)

    print(f"\n--- Cálculo de ROC y AUC finalizado ---")
    print(f"Área Bajo la Curva (AUC): {roc_auc:.4f}")

    # Selection of optimal thresholds
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    
    distances = np.sqrt(fpr**2 + (1 - tpr)**2)
    sorted_indices = np.argsort(distances)
    
    optimal_thresholds_for_plotting = [] 
    optimal_thresholds_for_metrics = [] 
    seen_thresholds_set = set() 

    if thresholds_roc_raw[best_idx] not in seen_thresholds_set:
        optimal_thresholds_for_metrics.append(thresholds_roc_raw[best_idx])
        optimal_thresholds_for_plotting.append(thresholds_roc_raw[best_idx])
        seen_thresholds_set.add(thresholds_roc_raw[best_idx])

    for idx in sorted_indices:
        current_threshold = thresholds_roc_raw[idx]
        if 0.001 < current_threshold < 0.999 and current_threshold not in seen_thresholds_set:
            optimal_thresholds_for_metrics.append(current_threshold)
            optimal_thresholds_for_plotting.append(current_threshold)
            seen_thresholds_set.add(current_threshold)
            if len(optimal_thresholds_for_metrics) >= 5: 
                break
    
    optimal_thresholds_for_metrics.sort() 
    optimal_thresholds_for_plotting.sort()
    
    print(f"\n--- 5 Umbrales 'Óptimos' detectados (basados en distancia a (0,1) en curva ROC o Youden's J) ---")
    if not optimal_thresholds_for_metrics:
        print("  No se pudieron encontrar 5 umbrales óptimos únicos en el rango (0,1) del mapa Mahalanobis.")
    for i, opt_thresh in enumerate(optimal_thresholds_for_metrics):
        idx = np.argmin(np.abs(thresholds_roc_raw - opt_thresh))
        print(f"  Umbral {i+1}: {opt_thresh:.4f} (TPR: {tpr[idx]:.4f}, FPR: {fpr[idx]:.4f})")
        
    roc_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'roc_curve_image_level.png')
    plot_roc_curve(fpr, tpr, roc_auc, optimal_thresholds_for_plotting, roc_save_path, thresholds_roc_raw)

    selected_threshold_for_eval = None
    if optimal_thresholds_for_metrics:
        selected_threshold_for_eval = 0.65#optimal_thresholds_for_metrics[len(optimal_thresholds_for_metrics) // 2]
        print(f"\n  Umbral seleccionado para visualización y métricas (mediana de umbrales óptimos): {selected_threshold_for_eval:.4f}")
    else:
        print("\nAdvertencia: No se encontraron umbrales óptimos. Usando un umbral por defecto de 0.5 para visualización y métricas.")
        selected_threshold_for_eval = 0.5 

    if selected_threshold_for_eval is None:
        print("No se pudo determinar un umbral para la evaluación. No se realizarán las visualizaciones, matriz de confusión ni tabla de métricas.")
        exit() 
    
    # --- Recalculate Predicted Labels for Metrics using the selected_threshold_for_eval ---
    recalculated_predicted_labels = []
    
    print(f"\n  Recalculando predicciones binarias para métricas con umbral {selected_threshold_for_eval:.4f} y MCC Area {MIN_CONNECTED_COMPONENT_AREA}...")
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        if not maps_list:
            continue
        for score_map in tqdm(maps_list, desc=f"    Applying threshold for {cls}"):
            binary_mask = apply_threshold_and_filter(score_map, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA)
            is_anomaly = classify_image_anomaly(binary_mask)
            recalculated_predicted_labels.append(1 if is_anomaly else 0)

    # --- PASO 5: Matriz de Confusión Global ---
    print("\n--- 5. Generando Matriz de Confusión Global ---")
    cm_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, f'confusion_matrix_thresh_{selected_threshold_for_eval:.4f}.png')
    plot_confusion_matrix(all_true_labels, recalculated_predicted_labels, cm_save_path, selected_threshold_for_eval)

    # --- PASO 6: Tabla de Métricas de Rendimiento y Guardar en Excel ---
    print("\n--- 6. Calculando, mostrando y guardando Tabla de Métricas de Rendimiento ---")
    # Pass AUC to the metrics function so it can be saved in the Excel file
    metrics_data = calculate_and_print_metrics(all_true_labels, recalculated_predicted_labels, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA, roc_auc)

    # Convert the metrics dictionary to a pandas DataFrame
    metrics_df = pd.DataFrame([metrics_data]) 
    
    # Define the Excel file path
    excel_save_path = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'image_level_metrics.xlsx')
    
    # Check if the file exists. If it does, append. Otherwise, create a new one.
    if os.path.exists(excel_save_path):
        try:
            existing_df = pd.read_excel(excel_save_path)
            # Ensure column order consistency
            for col in metrics_df.columns:
                if col not in existing_df.columns:
                    existing_df[col] = np.nan # Add new columns from new data if they don't exist
            
            combined_df = pd.concat([existing_df, metrics_df], ignore_index=True)
            combined_df.to_excel(excel_save_path, index=False)
            print(f"✅ Métricas añadidas al archivo Excel existente: {excel_save_path}")
        except Exception as e:
            print(f"⚠️ Error al añadir métricas al archivo Excel existente, creando uno nuevo. Error: {e}")
            metrics_df.to_excel(excel_save_path, index=False)
            print(f"✅ Métricas guardadas en un nuevo archivo Excel: {excel_save_path}")
    else:
        metrics_df.to_excel(excel_save_path, index=False)
        print(f"✅ Métricas guardadas en un nuevo archivo Excel: {excel_save_path}")

    # --- PASO 7. Generando visualizaciones de máscaras de anomalía para TODOS los mapas de Mahalanobis cargados ---
    print("\n--- 7. Generando visualizaciones de máscaras de anomalía para TODOS los mapas de Mahalanobis cargados ---")
    print(f"    (Las imágenes se guardarán en: {os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'overlays_all_images')})")

    # Create a specific subdirectory for the multiple visualizations
    overlays_save_dir = os.path.join(BASE_PLOT_SAVE_ROOT_DIR, 'overlays_all_images')
    os.makedirs(overlays_save_dir, exist_ok=True)

    total_visualizations = 0
    for cls in CLASSES:
        maps_list = normalized_mahalanobis_maps.get(cls, [])
        file_ids = MAP_FILE_IDS.get(cls, [])

        if not maps_list:
            continue

        print(f"  Procesando visualizaciones para la clase: '{cls}' ({len(maps_list)} imágenes)")
        for i, score_map in enumerate(tqdm(maps_list, desc=f"    Generando overlays para {cls}")):
            image_id = file_ids[i]
            original_image_path = os.path.join(BASE_IMAGE_DIR, cls, image_id + '.png') 
            
            if os.path.exists(original_image_path):
                save_viz_path = os.path.join(overlays_save_dir, f'overlay_{cls}_{image_id}_thresh_{selected_threshold_for_eval:.4f}.png')
                visualize_overlay(original_image_path, score_map, selected_threshold_for_eval, MIN_CONNECTED_COMPONENT_AREA, save_viz_path)
                total_visualizations += 1
            else:
                print(f"Advertencia: La imagen original no se encontró en {original_image_path}. No se generó visualización para esta.")
    
    print(f"\n¡Se generaron {total_visualizations} visualizaciones de máscaras de anomalía!")
    print("\n¡Proceso de evaluación completado!")

--- 1. Detectando clases y cargando mapas de Mahalanobis ---
  Clases detectadas: ['crack', 'cut', 'evaluacion_roc', 'good', 'hole', 'print']
  Total de mapas cargados para 'crack': 10
  Total de mapas cargados para 'cut': 10
Advertencia: No se encontraron archivos .npy para la clase 'evaluacion_roc' en /home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc
  Total de mapas cargados para 'good': 10
  Total de mapas cargados para 'hole': 10
  Total de mapas cargados para 'print': 10
--- Mapas cargados exitosamente ---

  Clases finales para procesamiento: ['crack', 'cut', 'good', 'hole', 'print']
--- 2. Calculando mínimos, máximos globales y promedio del top 1% ---
  Mínimo global (min_final): 0.0
  Máximo global (max_final): 316.0980224609375
  Umbral del percentil 99.0 (para el top 1.0%): 196.3979
  Promedio de valores en el top 1.0%: 221.8546
--- Cálculo de min/max globales y promedio del top 1% finalizado ---

--- 3. Normalizando mapas de Mahalanobis ---
--- Normalización de map

    Procesando mapas de print: 100%|██████████| 10/10 [00:00<00:00, 970.52it/s]


--- Cálculo de ROC y AUC finalizado ---
Área Bajo la Curva (AUC): 0.9550

--- 5 Umbrales 'Óptimos' detectados (basados en distancia a (0,1) en curva ROC o Youden's J) ---
  Umbral 1: 0.6704 (TPR: 0.9500, FPR: 0.2000)
  Umbral 2: 0.7558 (TPR: 0.8750, FPR: 0.2000)
  Umbral 3: 0.7682 (TPR: 0.8750, FPR: 0.1000)
  Umbral 4: 0.7929 (TPR: 0.8000, FPR: 0.1000)
  Umbral 5: 0.8329 (TPR: 0.8000, FPR: 0.0000)


✅ Curva ROC guardada en: /home/imercatoma/FeatUp/graficas_evaluacion/evaluacion_roc/roc_curve_image_level.png

  Umbral seleccionado para visualización y métricas (mediana de umbrales óptimos): 0.6500

  Recalculando predicciones binarias para métricas con umbral 0.6500 y MCC Area 100...


    Applying threshold for print: 100%|██████████| 10/10 [00:00<00:00, 29.49it/s]


...

  Procesando visualizaciones para la clase: 'hole' (10 imágenes)


    Generando overlays para hole: 100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


  Procesando visualizaciones para la clase: 'print' (10 imágenes)


    Generando overlays para print: 100%|██████████| 10/10 [00:15<00:00,  1.58s/it]


¡Se generaron 50 visualizaciones de máscaras de anomalía!

¡Proceso de evaluación completado!
